In [1]:
import pandas as pd
from fastai.text import * 
import os
import numpy as np

In [2]:
os.getcwd()

'/home/ubuntu/Austin_endoscopy'

In [3]:
filestem = 'Total_ECE_Copy.xlsx'

In [4]:
path = os.getcwd()

In [5]:
df = pd.read_excel(path+'/'+filestem,sheet_name=1)
df
columnsTitles=["Entire Colon Examined","Advanced To"]
df=df.reindex(columns=columnsTitles)
df

,Entire Colon Examined,Advanced To
0,1,the terminal ileum
1,1,the terminal ileum
2,1,the terminal ileum
3,1,the terminal ileum
4,1,the terminal ileum
5,1,"the caecum, identified by appendiceal orifice ..."
6,1,"the caecum, identified by appendiceal orifice ..."
7,1,the terminal ileum
8,1,"the caecum, identified by appendiceal orifice ..."
9,1,"the caecum, identified by appendiceal orifice ..."


In [6]:
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
valid = df[~msk]

In [7]:
data_lm = TextLMDataBunch.from_df(path, train_df = train, valid_df = valid)



In [8]:
data_clas = TextClasDataBunch.from_df(path, train_df = train, valid_df = valid, vocab=data_lm.train_ds.vocab, bs=32)

In [9]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [10]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,8.520858,7.221618,0.116295,00:01


In [11]:
learn.predict("the ileocolonic")

'the ileocolonic '

In [12]:
learn.save_encoder('ft_enc')

In [13]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [14]:
data_clas.show_batch()


text,target
"xxbos the terminal ileum , with identification of the appendiceal orifice and xxup ic valve",1
"xxbos the terminal ileum , with identification of the appendiceal orifice and xxup ic valve",1
"xxbos the terminal ileum , with identification of the appendiceal orifice and xxup ic valve",1
"xxbos the terminal ileum , with identification of the appendiceal orifice and xxup ic valve",1
"xxbos the terminal ileum , with identification of the appendiceal orifice and xxup ic valve",1


In [15]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.272039,0.256514,0.917466,00:01


In [16]:
learn.freeze_to(-2)

In [17]:
learn.predict("the ileocolonic anastomosis")

(Category 1, tensor(1), tensor([0.0508, 0.9492]))

In [18]:
txt_ci = TextClassificationInterpretation.from_learner(learn)

In [19]:
txt_ci.show_top_losses(50)


Text,Prediction,Actual,Loss,Probability
"xxbos the terminal ileum , with identification of the appendiceal orifice and xxup ic valve",1,1,3.38,0.97
"xxbos the caecum , identified by appendiceal orifice and ileocaecal valve",1,1,3.38,0.97
"xxbos the caecum , identified by appendiceal orifice and ileocaecal valve",1,1,3.30,0.96
xxbos the descending colon,1,0,3.30,0.04
xxbos the terminal ileum,1,1,3.29,0.96
xxbos the splenic flexure,1,0,3.29,0.04
xxbos the terminal ileum,1,1,2.98,0.95
xxbos the terminal ileum,1,1,2.98,0.95
"xxbos the caecum , identified by appendiceal orifice and ileocaecal valve",1,1,2.98,0.95
xxbos the terminal ileum,1,1,2.98,0.95


In [22]:
learn.predict("the terminal ileum")

(Category 1, tensor(1), tensor([3.1341e-04, 9.9969e-01]))